In [1]:
import numpy as np
import pandas as pd

In [2]:
db = pd.read_csv("Diagnosticus.csv", sep= ";", index_col = 0 )

In [3]:
db.head()

,X11,X12,X13,X21,X22,X23,X31,X32,X33,X34,...,X72,X73,X81,X82,X83,X91,X92,X93,X94,Unnamed: 31
Y001,0,1,0,1,0,0,0,1,0,0.0,...,0,1.0,0,0,1,0,0,0,1,P
Y002,0,1,0,0,1,0,1,0,0,0.0,...,1,0.0,0,0,1,1,0,0,0,D
Y003,1,0,0,0,0,1,0,0,1,0.0,...,1,0.0,1,0,0,0,0,1,0,M
Y004,0,0,1,0,0,1,0,0,0,0.0,...,1,0.0,0,1,0,0,0,0,1,S
Y005,0,0,1,0,0,1,0,0,0,0.0,...,0,1.0,0,1,0,0,0,0,1,S


In [4]:
db = db.replace('o', 0)
db = db.replace('u', 0)
db = db.replace('0_', 0)
db = db.replace('T', 0)
db = db.replace("0'", 0)

In [5]:
def criar_banco_cromossomos (numero, tamanho):
    banco = np.zeros([numero,tamanho])
    for i in range(numero):
        sort = np.random.uniform(low=-1, high=1.01, size= tamanho)
        banco[i] = sort
    return banco

In [6]:
def definir_pesos(cromos_fit):
    pesos_fit = np.zeros(cromos_fit.shape[0])
    for i in range(cromos_fit.shape[0]):
        pesos_fit[i] = cromos_fit[i] / cromos_fit.sum()
    return pesos_fit

In [7]:
def calcular_fit(cromossomo, base_dados):
    d_certos = p_certos = s_certos = g_certos = m_certos = d_total = p_total = s_total = g_total = m_total = 1
    c100 = np.zeros(100)
    #ipdb.set_trace()
    for i in range(100):
        s = int(np.random.randint(base_dados.shape[0], size=1))
        c100[i] = np.sum(pd.to_numeric(base_dados.iloc[s][:30]).values * cromossomo)
        if base_dados.iloc[s][-1] == "D":
            d_total += 1
            if c100[i] > 3:
                d_certos += 1
        elif base_dados.iloc[s][-1] == "P":
            p_total += 1
            if c100[i] > 1:
                p_certos += 1
        elif base_dados.iloc[s][-1] == "S":
            s_total += 1
            if c100[i] > -1:
                s_certos += 1
        elif base_dados.iloc[s][-1] == "G":
            g_total += 1
            if c100[i] > -3:
                g_certos += 1
        elif base_dados.iloc[s][-1] == "M":
            m_total += 1
            if c100[i] < -3:
                m_certos += 1
        else:
            print("errou")
    fit = (m_certos/m_total) + (g_certos/g_total) + (s_certos/s_total) + (g_certos/g_total) + (m_certos/m_total)
    return fit

In [8]:
def roda_acaso(pesos_fit):
    a = np.cumsum(pesos_fit)
    b = np.random.rand()
    for i in range(pesos_fit.shape[0]):
        if b <= a[i]:
            return i

In [9]:
def cross(vetor1, vetor2):
    s = int(np.random.randint(1, vetor1.size, size=1))
    x = vetor1[:s]
    y = vetor2[s:]
    return np.concatenate((x,y))

In [10]:
def x_men(vetor):
    s = np.random.randint(vetor.size, size=1)
    vetor[s] = np.random.uniform(low=-1, high=1.01, size=1)

In [11]:
def menor_fit(cromos):
    a = np.where(np.array(cromos) == np.array(cromos).min())[0][0]
    return cromos[a], a

In [12]:
cromos1 = criar_banco_cromossomos(20,30)
cromos1[0] = np.array([-0.81084794,  0.1905629 ,  0.918404  ,  0.90500639,  0.84200309,
        0.42340951, -0.87177353,  0.03618851, -0.90383741, -0.58684081,
        0.38408509,  0.15580805, -0.82661333,  0.70802569,  0.15132862,
       -0.06652324, -0.91074909, -0.11264349,  0.77206266, -0.96194849,
       -0.01037924,  0.333686  , -0.59782002, -0.83215105,  0.09046913,
        0.45284157,  0.45284157,  0.45977842, -0.971764  ,  0.45284157])
for ii in range(200):
    #import ipdb; ipdb.set_trace() #debugging starts here
    fits1 = np.zeros(cromos1.shape[0])
    pesos1 = np.zeros(cromos1.shape[0])
    for jj in range(cromos1.shape[0]):
        fits1[jj] = calcular_fit(cromos1[jj],db)
    pesos1 = definir_pesos(fits1)
    s1 = roda_acaso(pesos1)
    s2 = roda_acaso(pesos1)
    while s1 == s2:
        s2 = roda_acaso(pesos1)
    m, v = menor_fit(fits1)
    cs_filho = cross(cromos1[s1], cromos1[s2])
    if np.random.rand() > 0.5:
        x_men(cs_filho)
    cs_menor = m
    if calcular_fit(cs_filho,db) > calcular_fit(cromos1[v],db):
        cromos1[v] = cs_filho

In [13]:
for k in range(20):
    acertou = 0
    base_dados = db
    for s in range(db.shape[0]):
        if base_dados.iloc[s][-1] == "D":
            #d_total += 1
            if np.sum(pd.to_numeric(base_dados.iloc[s][:30]).values * cromos1[k]) > 3:
                acertou += 1
        elif base_dados.iloc[s][-1] == "P":
            #p_total += 1
            if np.sum(pd.to_numeric(base_dados.iloc[s][:30]).values * cromos1[k]) > 1:
                acertou += 1
        elif base_dados.iloc[s][-1] == "S":
            #s_total += 1
            if np.sum(pd.to_numeric(base_dados.iloc[s][:30]).values * cromos1[k]) > -1:
                acertou += 1
        elif base_dados.iloc[s][-1] == "G":
            #g_total += 1
            if np.sum(pd.to_numeric(base_dados.iloc[s][:30]).values * cromos1[k]) > -3:
                acertou += 1
        elif base_dados.iloc[s][-1] == "M":
            #m_total += 1
            if np.sum(pd.to_numeric(base_dados.iloc[s][:30]).values * cromos1[k]) < -3:
                acertou += 1
    acertos = acertou/db.shape[0]
    print(acertos)

0.7281795511221946
0.6758104738154613
0.7281795511221946
0.7281795511221946
0.6758104738154613
0.7281795511221946
0.7630922693266833
0.6758104738154613
0.6758104738154613
0.6758104738154613
0.7281795511221946
0.6758104738154613
0.7281795511221946
0.7281795511221946
0.7281795511221946
0.6758104738154613
0.6758104738154613
0.7231920199501247
0.6758104738154613
0.7281795511221946


In [14]:
cromos1[6]

array([-0.81084794,  0.1905629 ,  0.918404  ,  0.90500639,  0.84200309,
        0.42340951, -0.87177353,  0.98398324, -0.90383741, -0.58684081,
        0.38408509,  0.15580805, -0.82661333,  0.70802569,  0.15132862,
       -0.06652324, -0.91074909, -0.11264349,  0.77206266, -0.96194849,
       -0.01037924, -0.64011349, -0.59782002, -0.83215105,  0.09046913,
        0.45284157,  0.45284157,  0.45977842, -0.971764  ,  0.45284157])

In [15]:
# pesos com 76% de acerto:
cromo_campeao = np.array([-0.81084794,  0.1905629 ,  0.918404  ,  0.90500639,  0.84200309,
        0.42340951, -0.87177353,  0.98398324, -0.90383741, -0.58684081,
        0.38408509,  0.15580805, -0.82661333,  0.70802569,  0.15132862,
       -0.06652324, -0.91074909, -0.11264349,  0.77206266, -0.96194849,
       -0.01037924, -0.64011349, -0.59782002, -0.83215105,  0.09046913,
        0.45284157,  0.45284157,  0.45977842, -0.971764  ,  0.45284157])
cromo_campeao

array([-0.81084794,  0.1905629 ,  0.918404  ,  0.90500639,  0.84200309,
        0.42340951, -0.87177353,  0.98398324, -0.90383741, -0.58684081,
        0.38408509,  0.15580805, -0.82661333,  0.70802569,  0.15132862,
       -0.06652324, -0.91074909, -0.11264349,  0.77206266, -0.96194849,
       -0.01037924, -0.64011349, -0.59782002, -0.83215105,  0.09046913,
        0.45284157,  0.45284157,  0.45977842, -0.971764  ,  0.45284157])